In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch
from torch import nn
from torch.utils import data

In [2]:
# 以下为给琪琪的接口

In [3]:
pca_per_ppd = pd.read_csv('./pca_per.csv',index_col=0,header=None)
pca_job_ppd = pd.read_csv('./pca_job.csv',index_col=0,header=None)

In [4]:
pca_per_myd = pd.read_csv('./pca_per_myd.csv',index_col=0,header=None)
pca_job_myd = pd.read_csv('./pca_job_myd.csv',index_col=0,header=None)

In [5]:
def get_input_ppd(pid,rid):
  per = pca_per_ppd.loc[pid].values
  job = pca_job_myd.loc[rid].values
  all = np.concatenate([per,job],axis=0)
  return all

def get_input_myd(pid,rid):
  per = pca_per_myd.loc[pid].values
  job = pca_job_myd.loc[rid].values
  all = np.concatenate([per,job],axis=0)
  return all

In [6]:
net_ppd = torch.load('./Sigmoid-200.pt')
net_myd = torch.load('./Sigmoid-MYD.pt')

In [7]:
# 硬核条件，这里的序号是0~N-1
hd_job = pd.read_csv('./hardcore_position.csv',index_col=0,header=0,encoding='gbk')
hd_job.index = [i for i in range(hd_job.shape[0])]
hd_per = pd.read_csv('./hardcore_user.csv',index_col=0,header=0,encoding='gbk')
hd_per.index = [i for i in range(hd_per.shape[0])]
hd_job.shape, hd_per.shape

((1575, 4), (10877, 3))

In [8]:
# 属性嵌入，这里的序号是0~N-1
# 用于拼接向量 and 判断行业
a_emb_job = pd.read_csv('./attribute_embedding_position.txt',sep=' ',index_col=0,header=None,na_values=[])
a_emb_per = pd.read_csv('./attribute_embedding_user.txt',sep=' ',index_col=0,header=None,na_values=[])

In [9]:
def is_fw(pid):
  val = fw_list.loc[pid,'是否为废人']
  return val==1

fw_list = pd.read_csv('./fw.txt',sep=' ',index_col=0)

In [31]:
def judge_hangye(pid,rid): # 0~N-1
  # print(pid,rid)
  x = a_emb_per.loc[pid,6:14].values.astype('int')
  y = a_emb_job.loc[rid,6:14].values.astype('int')
  sum = (x*y).sum()
  return sum > 0

def judge_gongzi(pid,rid): # 0~N-1
  p2, p1 = hd_per.iloc[pid,:-1]
  r2, r1 = hd_job.iloc[rid,-2:]
  return max(p1,r1)<=min(p2,r2)

def judge_type(pid,rid):
  pt = hd_per.iloc[pid,-1]
  rt = hd_job.iloc[rid,0]
  # print(pt,rt)
  return pt == rt

def judge(pid,rid):
  if judge_buf[pid,rid]>=0:
    return judge_buf[pid,rid]==1
  # b1 = judge_hangye(pid,rid)
  b2 = judge_gongzi(pid,rid)
  b3 = judge_type(pid,rid)
  if b2 and b3:
    judge_buf[pid,rid] = 1
  else:
    judge_buf[pid,rid] = 0
  # print('build buf')
  return judge_buf[pid,rid]==1

judge_buf = np.zeros(shape=(10877,1575),dtype='int')-1
judge_buf

array([[-1, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1],
       ...,
       [-1, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1]])

In [14]:
ppd_buf = {}
myd_buf = {}
for i in pca_per_ppd.index:
  ppd_buf[i] = np.zeros(1575)-1
  myd_buf[i] = np.zeros(1575)-1

def get_ppd(pid,rid):
  if judge(pid,rid)==False: # 硬性条件不满足为0
    return 0
  elif is_fw(pid):  # 废物为1
    return 0.1
  elif ppd_buf[pid][rid]>=0:
    return ppd_buf[pid][rid]
  else:
    ppd = net_ppd(torch.Tensor(get_input_ppd(pid,rid))).item()
#     print(ppd)
    # print(f'ppd({pid},{rid})={ppd}')
  ppd = min(ppd,10)
  ppd = max(ppd,1)/10
  ppd_buf[pid][rid] = ppd
  return ppd

def get_myd(pid,rid):
  if judge(pid,rid)==False: # 硬性条件不满足为0
    return 0
  elif is_fw(pid):  # 废物为1
    return 0.1
  elif myd_buf[pid][rid]>=0:
    return myd_buf[pid][rid]
  else:
    myd = net_myd(torch.Tensor(get_input_myd(pid,rid))).item()
#     print(myd)
    # print(f'ppd({pid},{rid})={ppd}')
  myd = min(myd,10)
  myd = max(myd,1)/10
  myd_buf[pid][rid] = myd
  return myd

In [20]:
# 调用实例，使用时可以把函数里的print关掉
# 仅第一次调用神经网络，即建立缓存时会print
a,b = 94,879 # a是求职者，b是岗位
get_ppd(a,b),get_myd(a,b)

(0.7098597049713135, 0.7095348358154296)

In [13]:
# 接口到此为止

In [25]:
from tqdm import tqdm

In [2]:
user_map = pd.read_csv("map/user_map.csv")
position_map = pd.read_csv("map/position_map.csv")

In [4]:
user_map.head(),position_map.head()

(                求职者 ID
 0  1648621861036228608
 1  1648621300526219264
 2  1648345891960127488
 3  1648221000086716416
 4  1647944223779061760,
                招聘信息 ID  企业名称
 0  1648527394191052802   众云网
 1  1648527394191052801   众云网
 2  1648527394191052800   众云网
 3  1648165203084447745  森羽网络
 4  1648165203084447744  森羽网络)

In [38]:
ppd_list = []
for p_index,p_row in tqdm(position_map.iterrows()):
    cur = []
    for u_index,u_row in user_map.iterrows():
        ppd = get_ppd(u_index,p_index)
        if ppd>0:
            cur.append((p_row["招聘信息 ID"],u_row["求职者 ID"],ppd))
            cur.sort(key=lambda x:x[2],reverse=True)
#             print(cur)
    ppd_list += cur
#     print(ppd_list)

1575it [2:33:10,  5.84s/it]


In [40]:
ppd_df = pd.DataFrame(data=ppd_list,columns=["招聘信息 ID","求职者 ID","岗位匹配度"])
# ppd_df.sort_values(by=['岗位匹配度'], inplace=True)
ppd_df.to_csv("res/ppd.csv",index=0)

In [43]:
myd_list = []
for u_index,u_row in tqdm(user_map.iterrows()):
    cur = []
    for p_index,p_row in position_map.iterrows():
        myd = get_myd(u_index,p_index)
        if myd>0:
            cur.append((u_row["求职者 ID"],p_row["招聘信息 ID"],myd))
            cur.sort(key=lambda x:x[2],reverse=True)
    ppd_list += cur

3686it [12:00,  5.11it/s]


KeyboardInterrupt: 

In [ ]:
myd_df = pd.DataFrame(data=myd_list,columns=["求职者 ID","招聘信息 ID","求职者满意度"])
myd_df.contact(position_map,on="招聘信息 ID",how="left")
myd_df = myd_df[["求职者 ID","招聘信息 ID","企业名称","求职者满意度"]]
myd_df.rename(columns={'企业名称':'公司名称'},inplace = True)
# myd_df.sort_values(by=['求职者满意度'], inplace=True)
myd_df.to_csv("res/myd.csv",index=0)